In [793]:
from typing import List, Tuple, Dict
from copy import copy

from tqdm import tqdm_notebook
import numpy as np
import pandas as pd

In [794]:
backdoor = pd.read_csv('b_train.csv',header=None)

In [795]:
backdoor = backdoor.values

In [796]:
res_backdoor  = np.concatenate((backdoor, np.zeros((len(backdoor[:, -1]), 1))), axis=1)

In [797]:
clean = pd.read_csv('c_train.csv',header=None)
clean = clean.values

In [798]:
res_clean  = np.concatenate((clean,  np.ones((len(clean[: ,-1]), 1))), axis=1)

In [799]:
res_clean

array([[0.05376492, 0.05409903, 0.1359825 , ..., 0.11848674, 6.        ,
        1.        ],
       [0.02252906, 0.00967184, 0.20531535, ..., 0.02936194, 7.        ,
        1.        ],
       [0.22453262, 0.066879  , 0.06422424, ..., 0.14379416, 5.        ,
        1.        ],
       ...,
       [0.4341815 , 0.01277268, 0.11431133, ..., 0.08081577, 2.        ,
        1.        ],
       [0.01950057, 0.03573278, 0.17800894, ..., 0.28225106, 2.        ,
        1.        ],
       [0.11309104, 0.01487672, 0.07516164, ..., 0.04210447, 7.        ,
        1.        ]])

In [800]:
model_results = np.vstack((res_clean, res_backdoor))

In [801]:
model_results[:, -1]

array([1., 1., 1., ..., 0., 0., 0.])

In [802]:
model_results[:, -2]

array([6., 7., 5., ..., 2., 2., 7.])

In [803]:
model_results

array([[5.3764924e-02, 5.4099027e-02, 1.3598250e-01, ..., 1.1848674e-01,
        6.0000000e+00, 1.0000000e+00],
       [2.2529058e-02, 9.6718390e-03, 2.0531535e-01, ..., 2.9361939e-02,
        7.0000000e+00, 1.0000000e+00],
       [2.2453262e-01, 6.6879004e-02, 6.4224236e-02, ..., 1.4379416e-01,
        5.0000000e+00, 1.0000000e+00],
       ...,
       [7.0030540e-01, 2.7862402e-02, 5.6386855e-03, ..., 2.1589138e-02,
        2.0000000e+00, 0.0000000e+00],
       [3.8068290e-01, 1.4706442e-01, 4.3616480e-02, ..., 1.2089326e-01,
        2.0000000e+00, 0.0000000e+00],
       [3.9320430e-01, 1.1350253e-03, 6.9377600e-02, ..., 4.4483450e-02,
        7.0000000e+00, 0.0000000e+00]])

In [804]:
len(model_results)

478800

batch_size = 2000
stride = 9984

num_batches = (model_results.shape[0] - batch_size) // stride + 1

batch_list = []

for i in range(num_batches):
    start = i * stride
    end = start + batch_size
    batch = model_results[start:end, :]
    batch_list.append(batch)

results = np.vstack(batch_list)

In [805]:
model_results.shape

(478800, 12)

In [806]:
model_results

array([[5.3764924e-02, 5.4099027e-02, 1.3598250e-01, ..., 1.1848674e-01,
        6.0000000e+00, 1.0000000e+00],
       [2.2529058e-02, 9.6718390e-03, 2.0531535e-01, ..., 2.9361939e-02,
        7.0000000e+00, 1.0000000e+00],
       [2.2453262e-01, 6.6879004e-02, 6.4224236e-02, ..., 1.4379416e-01,
        5.0000000e+00, 1.0000000e+00],
       ...,
       [7.0030540e-01, 2.7862402e-02, 5.6386855e-03, ..., 2.1589138e-02,
        2.0000000e+00, 0.0000000e+00],
       [3.8068290e-01, 1.4706442e-01, 4.3616480e-02, ..., 1.2089326e-01,
        2.0000000e+00, 0.0000000e+00],
       [3.9320430e-01, 1.1350253e-03, 6.9377600e-02, ..., 4.4483450e-02,
        7.0000000e+00, 0.0000000e+00]])

In [807]:
from sklearn.ensemble import RandomForestClassifier
from mblearn import AttackModels

In [808]:
rf_attack = RandomForestClassifier(n_estimators=100)
attacker = AttackModels(target_classes=10, attack_learner=rf_attack)
attacker.fit(model_results)

KeyboardInterrupt: 

In [772]:
backdoor_test = pd.read_csv('b_train_test.csv',header=None)
backdoor_test = backdoor_test.values

In [773]:
clean_test = pd.read_csv('c_train_test.csv',header=None)
clean_test = clean_test.values

In [774]:
print(len(clean_test))

79800


batch_size = 2000
stride = 60000

num_batches = (backdoor_test.shape[0] - batch_size) // stride + 1

batch_list = []

for i in range(num_batches):
    start = i * stride
    end = start + batch_size
    batch = backdoor_test[start:end, :]
    batch_list.append(batch)

backdoor_test = np.vstack(batch_list)

batch_size = 2000
stride = 60000

num_batches = (clean_test.shape[0] - batch_size) // stride + 1

batch_list = []

for i in range(num_batches):
    start = i * stride
    end = start + batch_size
    batch = clean_test[start:end, :]
    batch_list.append(batch)

clean_test = np.vstack(batch_list)

In [775]:
backdoor_test_out = backdoor_test[:,:-1]
backdoor_test_label = backdoor_test[:,-1]
clean_test_out = clean_test[:,:-1]
clean_test_label = clean_test[:,-1]

In [776]:
backdoor_test_label = backdoor_test_label.astype(int)
clean_test_label = clean_test_label.astype(int)

In [777]:
clean_test_out

array([[0.08951516, 0.0718791 , 0.02650975, ..., 0.0730058 , 0.0305978 ,
        0.04365988],
       [0.24233079, 0.20517854, 0.05907395, ..., 0.06319578, 0.02326809,
        0.03479784],
       [0.3150563 , 0.06085425, 0.01849577, ..., 0.02055259, 0.04436581,
        0.01528853],
       ...,
       [0.15775234, 0.16401517, 0.525223  , ..., 0.00287453, 0.07978477,
        0.05411028],
       [0.01427697, 0.00357587, 0.8767877 , ..., 0.00777552, 0.01848793,
        0.0556218 ],
       [0.17044358, 0.11466277, 0.00156177, ..., 0.02463803, 0.01857577,
        0.00954938]])

In [778]:
clean_test_label.shape

(79800,)

In [779]:
print(len(clean_test))

79800


In [780]:
print(len(backdoor_test))

79800


In [781]:
mem_res_clean = attacker.predict(clean_test_out ,clean_test_label, batch=True)
mem_res_backdoor = attacker.predict(backdoor_test_out ,backdoor_test_label, batch=True)

In [782]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [783]:
y_pred = np.concatenate((np.argmax(mem_res_clean, axis=1), np.argmax(mem_res_backdoor, axis=1)))
y_true = np.concatenate((np.ones_like(clean_test_label), np.zeros_like(backdoor_test_label)))

In [784]:
y_pred

array([1, 0, 0, ..., 1, 0, 0])

In [785]:
accuracy_score(y_true, y_pred)

0.7760714285714285

In [786]:
precision_score(y_true, y_pred)

0.7079428005096984

In [787]:
confusion_matrix(y_true, y_pred)

array([[48858, 30942],
       [ 4797, 75003]])

In [788]:
recall_score(y_true, y_pred)

0.9398872180451128

In [789]:
f1_score(y_true, y_pred)

0.8075910522490511

In [790]:
y_true

array([1, 1, 1, ..., 0, 0, 0])

In [791]:
y_pred

array([1, 0, 0, ..., 1, 0, 0])

In [792]:
arr = y_pred.reshape(-1, 7980) 

for i, group in enumerate(arr):
    ones = np.sum(group == 1)
    zeros = np.sum(group == 0)

    print(f'model {i}:')
    print(f'Ones: {ones}')
    print(f'Zeros: {zeros}')

    if ones > zeros :
        print('Result: 1')
    else:  
        print('Result: 0')

model 0:
Ones: 4932
Zeros: 3048
Result: 1
model 1:
Ones: 7750
Zeros: 230
Result: 1
model 2:
Ones: 7526
Zeros: 454
Result: 1
model 3:
Ones: 7884
Zeros: 96
Result: 1
model 4:
Ones: 7906
Zeros: 74
Result: 1
model 5:
Ones: 7918
Zeros: 62
Result: 1
model 6:
Ones: 7824
Zeros: 156
Result: 1
model 7:
Ones: 7946
Zeros: 34
Result: 1
model 8:
Ones: 7763
Zeros: 217
Result: 1
model 9:
Ones: 7554
Zeros: 426
Result: 1
model 10:
Ones: 2923
Zeros: 5057
Result: 0
model 11:
Ones: 3380
Zeros: 4600
Result: 0
model 12:
Ones: 5077
Zeros: 2903
Result: 1
model 13:
Ones: 1998
Zeros: 5982
Result: 0
model 14:
Ones: 1730
Zeros: 6250
Result: 0
model 15:
Ones: 2115
Zeros: 5865
Result: 0
model 16:
Ones: 1859
Zeros: 6121
Result: 0
model 17:
Ones: 2049
Zeros: 5931
Result: 0
model 18:
Ones: 4995
Zeros: 2985
Result: 1
model 19:
Ones: 4816
Zeros: 3164
Result: 1
